In [3]:
#/usr/bin/env python 3.6
# Hyperparameter search for a best dropout value

import sklearn
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from keras import utils as np_utils
from keras.datasets import fashion_mnist
import numpy as np
from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras import optimizers

def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    import numpy as np
    from sklearn.model_selection import train_test_split
    tmp = np.load('mnist.npz')
    X = tmp['X']
    y = tmp['y']
    X, y = X[:1000], y[:1000]
    X = X.reshape(X.shape[0], 28, 28, 1)
    # Normalize
    X = X / 255.
    # number of unique classes
    num_classes = len(np.unique(y))
    y = y.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
    num_tot = y.shape[0]
    num_train = y_train.shape[0]
    num_test = y_test.shape[0]

    y_oh = np.zeros((num_tot, num_classes))
    y_oh[range(num_tot), y] = 1

    y_oh_train = np.zeros((num_train, num_classes))
    y_oh_train[range(num_train), y_train] = 1

    y_oh_test = np.zeros((num_test, num_classes))
    y_oh_test[range(num_test), y_test] = 1
    return X_train, y_oh_train, X_test, y_oh_test


def model(X_train, Y_oh_train, X_test, Y_oh_test):
    import keras
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras.optimizers import Adam
    import numpy as np

    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    # Max pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    # Max pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(10, activation='softmax'))
    
    adam = keras.optimizers.Adam(lr=0.00145)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=adam)

    model.fit(X_train, y_oh_train,
              epochs=20,
              batch_size=32,
              validation_data=(X_test, y_oh_test))
    score, acc = model.evaluate(X_test, y_oh_test, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=30,
                                          trials=Trials(),
                                          notebook_name = 'DPopt3')
    X_train, y_oh_train, X_test, y_oh_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_oh_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)


>>> Imports:
#coding=utf-8

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dropout_1': hp.uniform('Dropout_1', 0, 1),
    }

>>> Data
  1: 
  2: """
  3: Data providing function:
  4: 
  5: This function is separated from create_model() so that hyperopt
  6: won't reload data for each evaluation run.
  7: """
  8: import numpy as np
  9: from sklearn.model_selection import train_test_split
 10: tmp = np.load('mnist.npz')
 11: X = tmp['X']
 12: y = tmp['y']
 13: X, y = X[:1000], y[:1000]
 14: X = X.reshape(X.shape[0], 28, 28, 1)
 15: # Normalize
 16: X = X / 255.
 17: # number of unique classes
 18: num_classes = len(np.unique(y))
 19: y = y.astype(int)
 20: 
 21: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
 22: num_tot = y.shape[0]
 23: num_train = y_train.shape[0]
 24: num_test = y_test.shape[0]
 25: 
 26: y_oh = np.zeros((num_tot, num_classes))
 27: y_oh[range(num_tot), y] =

800/800 [==============================] - 1s 723us/step - loss: 0.9165 - acc: 0.7025 - val_loss: 0.7215 - val_acc: 0.8100
Epoch 4/20
800/800 [==============================] - 1s 752us/step - loss: 0.6948 - acc: 0.7725 - val_loss: 0.5575 - val_acc: 0.8600
Epoch 5/20
800/800 [==============================] - 1s 797us/step - loss: 0.6028 - acc: 0.8000 - val_loss: 0.4404 - val_acc: 0.8850
Epoch 6/20
800/800 [==============================] - 1s 1ms/step - loss: 0.5231 - acc: 0.8488 - val_loss: 0.4645 - val_acc: 0.8550
Epoch 7/20
800/800 [==============================] - 1s 899us/step - loss: 0.4515 - acc: 0.8675 - val_loss: 0.3543 - val_acc: 0.9100
Epoch 8/20
800/800 [==============================] - 1s 780us/step - loss: 0.3667 - acc: 0.8900 - val_loss: 0.3428 - val_acc: 0.9050
Epoch 9/20
800/800 [==============================] - 1s 953us/step - loss: 0.3284 - acc: 0.9113 - val_loss: 0.2778 - val_acc: 0.9250
Epoch 10/20
800/800 [==============================] - 1s 731us/step - loss

Epoch 2/20
800/800 [==============================] - 1s 1ms/step - loss: 1.3709 - acc: 0.5737 - val_loss: 1.1280 - val_acc: 0.7600
Epoch 3/20
800/800 [==============================] - 1s 1ms/step - loss: 0.9798 - acc: 0.6700 - val_loss: 0.9114 - val_acc: 0.7800
Epoch 4/20
800/800 [==============================] - 1s 1ms/step - loss: 0.8326 - acc: 0.7162 - val_loss: 0.7602 - val_acc: 0.8150
Epoch 5/20
800/800 [==============================] - 1s 1ms/step - loss: 0.7201 - acc: 0.7700 - val_loss: 0.6551 - val_acc: 0.8150
Epoch 6/20
800/800 [==============================] - 1s 1ms/step - loss: 0.6677 - acc: 0.7913 - val_loss: 0.6311 - val_acc: 0.8600
Epoch 7/20
800/800 [==============================] - 1s 1ms/step - loss: 0.6231 - acc: 0.8025 - val_loss: 0.5615 - val_acc: 0.8250
Epoch 8/20
800/800 [==============================] - 1s 1ms/step - loss: 0.6258 - acc: 0.7963 - val_loss: 0.5710 - val_acc: 0.8550
Epoch 9/20
800/800 [==============================] - 1s 1ms/step - loss: 0.

Epoch 2/20
800/800 [==============================] - 1s 742us/step - loss: 1.3800 - acc: 0.5350 - val_loss: 1.2643 - val_acc: 0.7300
Epoch 3/20
800/800 [==============================] - 1s 697us/step - loss: 1.0941 - acc: 0.6525 - val_loss: 1.0696 - val_acc: 0.7650
Epoch 4/20
800/800 [==============================] - 1s 739us/step - loss: 0.9015 - acc: 0.7087 - val_loss: 0.9726 - val_acc: 0.7850
Epoch 5/20
800/800 [==============================] - 1s 741us/step - loss: 0.7759 - acc: 0.7512 - val_loss: 0.8455 - val_acc: 0.8100
Epoch 6/20
800/800 [==============================] - 1s 690us/step - loss: 0.7447 - acc: 0.7600 - val_loss: 0.7771 - val_acc: 0.8250
Epoch 7/20
800/800 [==============================] - 1s 684us/step - loss: 0.6615 - acc: 0.7888 - val_loss: 0.6222 - val_acc: 0.8600
Epoch 8/20
800/800 [==============================] - 1s 688us/step - loss: 0.6172 - acc: 0.8275 - val_loss: 0.6075 - val_acc: 0.8850
Epoch 9/20
800/800 [==============================] - 1s 711us

Epoch 2/20
800/800 [==============================] - 1s 788us/step - loss: 0.9540 - acc: 0.7037 - val_loss: 0.6840 - val_acc: 0.8050
Epoch 3/20
800/800 [==============================] - 1s 758us/step - loss: 0.5969 - acc: 0.8125 - val_loss: 0.5085 - val_acc: 0.8450
Epoch 4/20
800/800 [==============================] - 1s 816us/step - loss: 0.4811 - acc: 0.8463 - val_loss: 0.4435 - val_acc: 0.8600
Epoch 5/20
800/800 [==============================] - 1s 751us/step - loss: 0.3840 - acc: 0.8775 - val_loss: 0.3661 - val_acc: 0.8900
Epoch 6/20
800/800 [==============================] - 1s 813us/step - loss: 0.3137 - acc: 0.9025 - val_loss: 0.3285 - val_acc: 0.9050
Epoch 7/20
800/800 [==============================] - 1s 753us/step - loss: 0.2806 - acc: 0.9050 - val_loss: 0.3086 - val_acc: 0.9100
Epoch 8/20
800/800 [==============================] - 1s 783us/step - loss: 0.2529 - acc: 0.9237 - val_loss: 0.2971 - val_acc: 0.9200
Epoch 9/20
800/800 [==============================] - 1s 722us

800/800 [==============================] - 4s 5ms/step - loss: 1.9014 - acc: 0.3825 - val_loss: 1.2513 - val_acc: 0.6600
Epoch 2/20
800/800 [==============================] - 1s 759us/step - loss: 0.7835 - acc: 0.7650 - val_loss: 0.6631 - val_acc: 0.7750
Epoch 3/20
800/800 [==============================] - 1s 725us/step - loss: 0.5135 - acc: 0.8313 - val_loss: 0.4411 - val_acc: 0.8750
Epoch 4/20
800/800 [==============================] - 1s 776us/step - loss: 0.3464 - acc: 0.8950 - val_loss: 0.3350 - val_acc: 0.9250
Epoch 5/20
800/800 [==============================] - 1s 811us/step - loss: 0.2899 - acc: 0.9213 - val_loss: 0.3076 - val_acc: 0.9050
Epoch 6/20
800/800 [==============================] - 1s 880us/step - loss: 0.2242 - acc: 0.9462 - val_loss: 0.2435 - val_acc: 0.9350
Epoch 7/20
800/800 [==============================] - 1s 800us/step - loss: 0.1569 - acc: 0.9575 - val_loss: 0.2598 - val_acc: 0.9400
Epoch 8/20
800/800 [==============================] - 1s 1ms/step - loss: 0

800/800 [==============================] - 4s 5ms/step - loss: 2.1312 - acc: 0.2438 - val_loss: 1.9266 - val_acc: 0.6500
Epoch 2/20
800/800 [==============================] - 1s 712us/step - loss: 1.5434 - acc: 0.4863 - val_loss: 1.2596 - val_acc: 0.6950
Epoch 3/20
800/800 [==============================] - 1s 717us/step - loss: 1.1561 - acc: 0.6150 - val_loss: 1.0051 - val_acc: 0.7700
Epoch 4/20
800/800 [==============================] - 1s 726us/step - loss: 0.9411 - acc: 0.6825 - val_loss: 0.7896 - val_acc: 0.7850
Epoch 5/20
800/800 [==============================] - 1s 712us/step - loss: 0.8269 - acc: 0.7275 - val_loss: 0.5922 - val_acc: 0.8350
Epoch 6/20
800/800 [==============================] - 1s 733us/step - loss: 0.7052 - acc: 0.7863 - val_loss: 0.5537 - val_acc: 0.8450
Epoch 7/20
800/800 [==============================] - 1s 724us/step - loss: 0.5929 - acc: 0.7900 - val_loss: 0.4851 - val_acc: 0.8350
Epoch 8/20
800/800 [==============================] - 1s 723us/step - loss:

In [4]:
print(best_model.evaluate(X_test,y_oh_test))

200/200 [==============================] - 0s 533us/step
[0.2124136596918106, 0.955]


In [ ]:
# the close-to-optimal hyperparameters for dropout1 is 0.2886625
# dropout2 is 0.337626